In [1]:
%load_ext autoreload

%autoreload 2
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.callbacks as cb
import time
import numpy as np
from data import get_datasets
import matplotlib.pyplot as plt
import os
import shutil
from functools import partial
from data import get_datasets
from experiment import create_model, evaluate, find_adv_k, sentiment, decode_sentence, visualize
import model_adv

In [2]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [3]:
def pick_sentence(max_len=40):
    # Find a small sentence for use in visualizations
    (train_dataset, test_dataset), info = get_datasets(batch_size=1)
    encoder = info.features['text'].encoder
    vocab_size=info.features['text'].encoder.vocab_size 

    it = iter(test_dataset)
    
    l = max_len
    x, y = next(it)
    _, l = x.shape

    while l > max_len:
        x, y = next(it)
        _, l = x.shape

    sentence_str, tokens = decode_sentence(x[0], encoder)
    print(l)
    print(sentence_str)
    return x[0], sentence_str, tokens

In [4]:
(train_dataset, test_dataset), info = get_datasets(batch_size=64)
encoder = info.features['text'].encoder
vocab_size=info.features['text'].encoder.vocab_size 

In [5]:
transformer, _optimizer, _checkpoint, _manager = create_model(models_dir="models/pos_enc_True", load_checkpoint=True, 
                           vocab_size=vocab_size, use_positional_encoding=True, run_eagerly=True)

Loaded previously trained model.


In [6]:
adv_model, adv_optimizer = model_adv.create_model(models_dir="adv", load_checkpoint=False, 
                                            run_eagerly=True, d_model=128)


In [8]:
# One epoch
for (x,y) in train_dataset:
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
                    adv_model, adv_optimizer, epochs=1)

In [9]:
x0, sentence_str, tokens = pick_sentence(max_len=40)

sent_original, weights_original = sentiment(sentence_str, encoder, transformer)
sent,          weights_adv    = sentiment(sentence_str, encoder=encoder, 
                      adv_model=adv_model, transformer=transformer)

print(f"Initial sentiment: {sent_original}")
print(f'Adversarial sentiment: {sent}')

23
A touching movie. It is full of emotions and wonderful acting. I could have sat through it a second time.
Initial sentiment: pos
Adversarial sentiment: pos


In [13]:
# Visualize
# visualize(tokens, weights_original[0])
visualize(tokens, weights_adv[0])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>